In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq as hq
import datetime as dt
import time
import warnings
import pmdarima as pm
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)

# Data cleaning and adding 
- start_time
- date
- Interval time

In [2]:
trace = pd.read_csv('Trace2021/AzureFunctionsInvocationTraceForTwoWeeksJan2021.txt')
trace["start_timestamp"] = trace["end_timestamp"] - trace["duration"]
trace = trace[['app','func','start_timestamp','end_timestamp','duration']]
trace = trace.rename(columns = {'app': 'app_id', 'func': 'func_id'}, inplace = False)
trace.sort_values(by=['start_timestamp'], ascending=False)

#append the date to the DF
newDate = []
for i in range(0,len(trace)):
    sd = dt.datetime(2021,1,31,0,0,0) # used to output incremental timestamp
    sd = sd + dt.timedelta(seconds = trace['start_timestamp'][i])
    #print(sd)
    newDate.append(sd)
    
#append IT to the DF
listIT = []
tempDict = {}
app_id = trace['app_id']
start_time = trace['start_timestamp']
for i in range(0,len(trace)):
    if tempDict.get(app_id[i]) == None:
        tempDict[app_id[i]] = start_time[i]
        listIT.append(0)
    else:
        IT = start_time[i] - tempDict.get(app_id[i])
        IT = np.floor(IT/60)
        tempDict[app_id[i]] = start_time[i]
        listIT.append(IT)
trace["date"] = newDate
trace["IT"] = listIT
trace.head()

,app_id,func_id,start_timestamp,end_timestamp,duration,date,IT
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,0.001491,0.079491,0.078,2021-01-31 00:00:00.001491,0.0
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,0.003860,57.157860,57.154,2021-01-31 00:00:00.003860,0.0
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,0.005478,59.130478,59.125,2021-01-31 00:00:00.005478,0.0
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,0.016541,6.252541,6.236,2021-01-31 00:00:00.016541,0.0
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,0.016824,0.066824,0.050,2021-01-31 00:00:00.016824,0.0


In [51]:
applicationCounts = trace['app_id'].value_counts().rename_axis('unique_application_id').reset_index(name='counts')
applicationCounts.head(10)

,unique_application_id,counts
0,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,535667
1,a9644719e5d173a27e098cffdd5e9227ba9c4a325d082c...,328885
2,96149d3ed4f00afb92f12856101e693e93c4683f030ae6...,299817
3,7958f89676bf3653f72cda8f38f6b3c0574dcae5169824...,143521
4,62ed48c098820db02aa8e99ad41e5438e61334ba7b1618...,124927
5,70b9cea7ca266637479483f517194c402dfe99b5fc2357...,71837
6,5fb02cfeb979fec16df02d1ece227157238477b19fc6e3...,59798
7,73e9cf25a1a3b312f60ead2177d7d259f128661e3b6763...,34713
8,6e42a4e8ea1df408e059ba37d0e04255fd8385235b3102...,31260
9,734272c01926d19690e5ec308bab64ef97950b75b1c758...,31100


# implementing histogram
- each application is stored in a dictionary as a key
- each key has a list of idle time (IT), with the first position stored as the last invocation time
- histogram will be plotted for every invidual key with bins of 0 to 240 (each bins are of 1mins total 4hour)

In [33]:
#sample data 1: to reflect incoming invocation call
#sampleTest = trace[['app_id','start_timestamp']].head(198095) #around 2 days
sampleTest = trace[['app_id','start_timestamp']].head(500)
#sampleTest = trace[['app_id','start_timestamp']]
incoming_invocations = sampleTest.values.tolist()
#print(sampleTest.tail(10))

In [4]:
incoming_invocations = [['app1', 1], # cold start, new id
['app1', 65], # cold start, IT = 1min, keep alive = current time + 1min
['app1', 130], # cold start, IT = 1min, keep alive 1min = current time + 1min
['app1', 150]] # warm start, 

In [7]:
incoming_invocations = [['app1', 1], # cold start, new id
['app1', 2], # cold start, IT = 1min, keep alive = current time + 1min
['app1', 3], # cold start, IT = 1min, keep alive 1min = current time + 1min
['app1', 4],
['app2', 10],
['app2', 30],
['app2', 40],
['app2', 200]  ,
['app2', 200]  ,
['app2', 220]  ,
['app2', 210]  ,
['app2', 250]  ,
['app2', 280]  ,
['app2', 300]  ,
['app1', 305]  ,
['app1', 310]     
                       ] # warm start, 

# how an application will be stored in a dictionary: 
{ 'application1': [datetime.datetime(2023, 3, 12, 2, 0, 22, 60499),  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0], <br>
 'application2': [datetime.datetime(2023, 3, 12, 2, 0, 22, 60499),  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0] }
- key: app_id
- value: list of idle time (IT) for every invocation (first position of the list contains the most recent invocation time)
- first element of the value is the latest function invocation time


# keep_alive_list parameters:
Priority heap queue: [(12.5, "app1"), (100, "app2"), (150, "app3")]
- (keep alive until, application_id)


In [32]:
# dictionary to store all the app_id
applicationDict = {}
# list to store all the current keep alive application id
keep_alive_list = []

In [34]:
default_keep_alive_time = 10
OOB_threshold = 5

# when an incoming function has been invoked
def function_invoked(app_id):
    # application id has not been seen before
    if applicationDict.get(app_id) == None:
        print("NEW app_id: {}".format(app_id))
        result.write("NEW app_id: {} \n".format(app_id))
        applicationDict[app_id] = []
        applicationDict[app_id].append(dt.datetime.now())
        print("COLD START BECAUSE OF NEW APPLICATION")
        result.write("COLD START BECAUSE OF NEW APPLICATION \n")
        return default_keep_alive_time
    
    # application id is already in the dictionary
    else:
        print("OLD app_id: {} ".format(app_id))
        result.write("OLD app_id: {} \n".format(app_id))
        curTime = dt.datetime.now()
        # calculate idle time (IT) by datetime.now - last previous invoked timing
        IT = curTime - applicationDict[app_id][0]
        IT = np.floor(IT.seconds/60)
        print('IDLE TIME (IT) for application: {} is: {} mins'.format(app_id,IT))
        result.write('IDLE TIME (IT) for application: {} is: {} \n'.format(app_id,IT))
        #add idle time into list
        applicationDict[app_id].append(IT)
        applicationDict[app_id][0] = curTime
        # if the idle time is more than 240mins, return predicted time as it is out of bounds (OOB)
        if(IT > 240):
            print('application is OOB with idle time(IT)')
            return predictIntervalTiming(app_id)   
        # add the IT into the dictionary and plot the histogram to determine the TTL duration (mins)
        else:
            return plotHistogram(app_id)
            
# hybrid histogram policy
# categorise the histogram into 240 different minutes bins, returning the bin with the highest amount of frequency
# in the event of same frequency, return the lowest number bin
def plotHistogram(app_id):
    appData = applicationDict[app_id][1:]
    print('appplication IT data is: {}'.format(appData))
    
    # Plot histogram with 240 bins ranging from 0 to 240
    counts, bins, _ = plt.hist(appData, bins=240, range=(0, 240), density=False, edgecolor='black')
    
    #check if histogram is representative
    CV = calculate_CV(counts)
    print('calculating CV = ',CV)
    if CV < 0.1:
        print('Histogram is not representative')
        return default_keep_alive_time
    print('Histogram is representative')
    # Add labels and title
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram')

    # Show the plot
    #plt.show()
    plt.close()
    print(counts)
    data = counts.tolist()
    #print("highest frequency of minutes bin of the application is: {}".format(data.index(max(data))))
    return data.index(max(data))

def update_heap_element(keep_alive_list, new_time, app_id):
    # Find the index of the element you want to update
    index = next((i for i, (value1, value2) in enumerate(keep_alive_list) if value2 == app_id), None)

    if index is not None:
        # Update the value of the element
        keep_alive_list[index] = (new_time, keep_alive_list[index][1])
        # Rebuild the heap
        hq.heapify(keep_alive_list)
        keep_alive_list.sort()
        print("WARM START")
        result.write("WARM START \n")
    else:
        hq.heappush(keep_alive_list, (new_time, app_id))
        print("COLD START")
        result.write("COLD START \n")

# Welford’s online algorithm (CV) to determine wheather a histogram is representive or not
def calculate_CV(app_bins):
    N = len(app_bins)
    M = 0
    S = 0

    for k in range(1, N+1):
        x = app_bins[k-1]
        oldM = M
        M = M + (x - M) / k
        S = S + (x - M) * (x - oldM)
    
    return S / (N - 1)

#prediction of application IT if too many OOB
#if number of OOB is below threshold, return standard keepalive time to save cost = TTL 10mins
def predictIntervalTiming(app_id):
    #do prediction of OOB applications here
    appData = applicationDict[app_id][1:]
    if len(appData) < OOB_threshold:
        print('Not enough OOB threshold of:,',OOB_threshold,'returning default time =', default_keep_alive_time)
        return default_keep_alive_time
    model1 = pm.auto_arima(appData,stepwise= False, seasonal= False, suppress_warnings= True)
    option1_predict = model1.predict(n_periods=1)
    print('application OOB predicted time = ',option1_predict)
    return option1_predict[0]
    
def addKeepAlive(app_id,TTL):
    return 0

def removeKeepAlive(app_id):
    return 0

def checkKeepAlive():
    return 0

# simulation
in this simulation, we have 2 kinds of timer, difference_timer and delta_timer.
- difference_timer: incrementally increasing in time since the start for time calculation
- delta_timer: printing of simulation's current time (every 1 second)

In [35]:
import datetime as dt
import time

start_time = dt.datetime.now()
timer = dt.datetime(2021,1,31,0,0,0) # used to output incremental timestamp
timer_temp = dt.datetime.now() # used to check for every second
result = open("result.txt","w") # output logs into a txt file as data is large
stop_time_seconds = 28800 # how long should the program run

count_warm = 0
count_cold = 0
count_oob = 0
count_oob_default = 0
count_oob_prediction = 0


while True:
    # accumulating seconds to track to check for incoming functions as well as calculating time to live (TTL)
    difference_timer = dt.datetime.now()-start_time 
    
    #non-incremental timer to check if 1sec has lapsed
    delta_timer = dt.datetime.now()-timer_temp
    
    #print out every second incrementally
    if delta_timer.seconds >= 1:
        timer = timer + dt.timedelta(0,1)
        print(timer.time())
        result.write("{}\n".format(timer.time()))
        timer_temp = dt.datetime.now()
    
    #if still have functions pending invocation
    if len(incoming_invocations) != 0:
        #while(True):
        # since the start_time column is sorted, the first index is always the first to be invoked
        incoming_application_time = dt.timedelta(seconds=incoming_invocations[0][1])
        if(difference_timer >= incoming_application_time):
        
            print("TRIGGERED: application has been invoked")
            result.write("TRIGGERED: application has been invoked \n")
            # here the return is the histogram output, 10 if OOB, -1 if new function
            keep_alive_time = function_invoked(incoming_invocations[0][0])
            # here we ignore -1 as a new function does not have any frequency
            # we calculate the TTL end time with current simulation time (difference_timer) + returned minute bin
            if(keep_alive_time>0):
                print("keep alive time is: {} mins".format(keep_alive_time))
                result.write("keep alive time is: {} mins \n".format(keep_alive_time))
                alive_timer = difference_timer + dt.timedelta(minutes = keep_alive_time)
                print("keep alive till time: {}".format(alive_timer.total_seconds()))
                #hq.heappush(keep_alive_list,(alive_timer.total_seconds(),incoming_invocations[0][0]))
                update_heap_element(keep_alive_list, alive_timer.total_seconds(), incoming_invocations[0][0])
                print("keep alive list: {}".format(keep_alive_list))
                result.write("keep alive list: {} \n".format(keep_alive_list))
            del incoming_invocations[0]
            
    
    # as long as there is still applications in the TTL list, we will keep checking
    if len(keep_alive_list) != 0:
        # since the TTL list is a heap queue, the first position is always the earliest terminated one
        # so we check if first element is pending expiry, then remove from heap queue
        check_expiry = keep_alive_list[0][0]
        # check if current simulation time has caught up with the application's TTL timing
        if difference_timer.total_seconds() >= check_expiry:
            print("{} container has expired".format(keep_alive_list[0][1]))
            result.write("{} container has expired \n".format(keep_alive_list[0][1]))
            hq.heappop(keep_alive_list)
            print(keep_alive_list)
            result.write("keep alive list: {} \n".format(keep_alive_list))
        #else:
            #print("{} container has NOT expired yet".format(keep_alive_list[0][1]))

    #terminate program if exceeded input time
    if (dt.datetime.now() - start_time).seconds >= stop_time_seconds:
        break
    if (len(incoming_invocations) == 0) and (len(keep_alive_list) == 0):
        break
    time.sleep(0.5)
        
print("==end of run==")
result.write("==end of run==\n")
result.close()

TRIGGERED: application has been invoked
NEW app_id: 7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de05270735055950a568
COLD START BECAUSE OF NEW APPLICATION
keep alive time is: 10 mins
keep alive till time: 600.001995
COLD START
keep alive list: [(600.001995, '7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de05270735055950a568')]
TRIGGERED: application has been invoked
NEW app_id: 1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556
COLD START BECAUSE OF NEW APPLICATION
keep alive time is: 10 mins
keep alive till time: 600.514894
COLD START
keep alive list: [(600.001995, '7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de05270735055950a568'), (600.514894, '1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556')]
00:00:01
TRIGGERED: application has been invoked
OLD app_id: 1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556 
IDLE TIME (IT) for application: 1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556 is: 0.0 mins
appplication IT data is: [0

TRIGGERED: application has been invoked
OLD app_id: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 
IDLE TIME (IT) for application: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 is: 0.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
calculating CV =  0.20416666666666683
Histogram is representative
[7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

00:00:11
TRIGGERED: application has been invoked
NEW app_id: 17c37a0fdd5d1932b755c0e6447137bc08fd524f455e14fdac414f584de08dc5
COLD START BECAUSE OF NEW APPLICATION
keep alive time is: 10 mins
keep alive till time: 612.860559
COLD START
keep alive list: [(601.628682, 'f274d71de386ccc77e4ca74766dbc485461c3053059d47266463c45ec92001b3'), (602.132497, '7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de05270735055950a568'), (602.813517, 'f7bfe5bc8d2a37a5c15986fbfc2c477a746e866adcb9663f9df7535b61c3eb9b'), (605.653463, '85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031'), (608.215058, '938e7f49544b3293cd6cc7ec3e63e1751085cf5cb6a004dcc9e94543934f607b'), (609.849253, '18ed3ca44bd1f7d411f1d047ed8cf38853fb184196afa59e91e68e5d06fda834'), (611.558526, '734272c01926d19690e5ec308bab64ef97950b75b1c7582283e0783fce1751d8'), (612.265675, '1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556'), (612.860559, '17c37a0fdd5d1932b755c0e6447137bc08fd524f455e14fdac414f584de08dc5')]
TRIGGERE

TRIGGERED: application has been invoked
OLD app_id: 734272c01926d19690e5ec308bab64ef97950b75b1c7582283e0783fce1751d8 
IDLE TIME (IT) for application: 734272c01926d19690e5ec308bab64ef97950b75b1c7582283e0783fce1751d8 is: 0.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
calculating CV =  0.5041666666666668
Histogram is representative
[11.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

00:00:20
TRIGGERED: application has been invoked
OLD app_id: 734272c01926d19690e5ec308bab64ef97950b75b1c7582283e0783fce1751d8 
IDLE TIME (IT) for application: 734272c01926d19690e5ec308bab64ef97950b75b1c7582283e0783fce1751d8 is: 0.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
calculating CV =  1.35
Histogram is representative
[18.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.

TRIGGERED: application has been invoked
OLD app_id: 1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556 
IDLE TIME (IT) for application: 1573b95c039e51cc012b543a4af3bc7c3ee9485acbb0033ba5648b74969e0556 is: 0.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0]
calculating CV =  0.1041666666666666
Histogram is representative
[5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

TRIGGERED: application has been invoked
OLD app_id: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 
IDLE TIME (IT) for application: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 is: 1.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
calculating CV =  0.20808926080892595
Histogram is representative
[7. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

TRIGGERED: application has been invoked
OLD app_id: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 
IDLE TIME (IT) for application: 85479ef37b5dc75dd5aeca3bab499129b97a134dac5d740d2c68941de9d63031 is: 0.0 mins
appplication IT data is: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
calculating CV =  0.34135285913528574
Histogram is representative
[9. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

KeyboardInterrupt: 

In [ ]:
for key,value in may28_3hr_dict.items():
    print(key)

In [17]:
applicationDict

{'app1': [datetime.datetime(2023, 6, 17, 15, 51, 9, 883144), 0.0, 0.0, 0.0]}

In [44]:
applicationDict['app1'][1:]


[0.0,
 0.0,
 0.0,
 9.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 64.0,
 1.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0]

In [49]:
a = predictIntervalTiming("app1")
print(a[0])

triggered oob prediction =  [5.51902583]
5.51902583139456


# test CV

In [ ]:
sample_bin1 = [0] * 240
sample_bin2 = [0] * 240
sample_bin1[1] = 3
sample_bin2[1] = 20

In [ ]:
print("sample_bin1 CV = {}".format(calculate_CV(sample_bin1)))
print("sample_bin2 CV = {}".format(calculate_CV(sample_bin2)))

In [ ]:
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Load/split your data
y = pm.datasets.load_wineind()
train, test = train_test_split(y, train_size=150)

# Fit your model
model = pm.auto_arima(train, seasonal=True, m=12)

# make your forecasts
forecasts = model.predict(test.shape[0])  # predict N steps into the future

# Visualize the forecasts (blue=train, green=forecasts)
x = np.arange(y.shape[0])
plt.plot(x[:150], train, c='blue')
plt.plot(x[150:], forecasts, c='green')
plt.show()

# testing for auto arima

In [ ]:
#traces of IT > 240
df = trace[['date','IT']]
df.set_index('date',inplace=True)
IT_more_than_240 = df.loc[df['IT'] > 240]
#IT_more_than_240.set_index('date',inplace=True)

print("{} of IT > 240".format(len(IT_more_than_240)))
IT_more_than_240.head()

In [ ]:
#traces of IT < 240
IT_less_than_240 = df.loc[df['IT'] <= 240]
#IT_less_than_240.set_index('date',inplace=True)

print("{} of IT <= 240".format(len(IT_less_than_240)))
IT_less_than_240.head()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1, 3,sharex=True, sharey=True, figsize=(15, 5))
fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')
ax1.set_xlabel('Normal IT')
ax2.set_xlabel('IT < 240')
ax3.set_xlabel('IT > 240')

ax1.plot(df)
ax2.plot(IT_less_than_240)
ax3.plot(IT_more_than_240)

In [ ]:
start = len(option1_train)
end = len(option1_train) + len(option1_test) -1
option1_predict = model1.predict(n_periods=len(option1_test))

option1_df = app
option1_predict.index = option1_df.index[start:end+1]
option1_df['forecast'] = [None]*len(option1_train) + list(option1_predict)

#clearer_image = option1_df.iloc[-1386665:]

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')


ax1.plot(option1_df)
#ax2.plot(clearer_image)

In [ ]:
from sklearn import metrics

print("Mean:",option1_test['IT'].mean())
#print("MSE:",metrics.mean_squared_error(option2_test['IT'],option2_predict))
print("RMSE:",np.sqrt(metrics.mean_squared_error(option1_test['IT'],option1_predict)))